<a href="https://colab.research.google.com/github/arikaran007/ML-Prediction/blob/main/spam_detector/Spam_Detector_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation

from sklearn.model_selection import StratifiedKFold

In [ ]:
df = pd.concat([pd.read_csv('/content/Youtube01-Psy.csv'),
                pd.read_csv('/content/Youtube02-KatyPerry.csv'),
                pd.read_csv('/content/Youtube03-LMFAO.csv'),
                pd.read_csv('/content/Youtube04-Eminem.csv'),
                pd.read_csv('/content/Youtube05-Shakira.csv')])


In [ ]:
df =df.sample(frac=1)

In [ ]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
40,z13vg3dgkqjycpttg23bvl2pwzzzwnteo,Ciasteczko ;*,2015-05-26T17:56:36.413000,LMFAO best songs ever!﻿,0
53,z12ujdnhpwroej3rd04cgrlguunjdnfwmkc0k,Bo Lanciné MAGASSOUBA,2015-05-28T11:47:37.722000,"WE GO FOR 1,000,000,000 FOR EMINEM﻿",0
111,z13mxjrbuyv5slha204cjdwg0wauylyap3w,Peter Wilkes,2014-09-15T15:48:57,"really want this video to get 1 billion views,...",0
62,z12uvzpw4tufgf2qw04chd5abu3uvf4rw5o,Eric Gonzalez,2015-05-24T15:35:03.844000,To help shakira become the first female to hit...,0
129,z13wtxw41lucynljt04cits4yxmsht0qjrg0k,Marshmallow Kingdom,2015-05-20T12:38:04.230000,BEST PARTY SONG LITERALLY PARTY ROCK IS IN THE...,0


In [ ]:
kfold = StratifiedKFold(n_splits=5)
splits = kfold.split(df,df['CLASS'])

In [ ]:
for train, test in splits:
  print("splits")
  print(test)

splits
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

In [ ]:
def train_test(train_idx,test_idx):
  train_cont = df['CONTENT'].iloc[train_idx]
  test_cont = df['CONTENT'].iloc[test_idx]

  token = Tokenizer(num_words=1000)

  token.fit_on_texts(train_cont)

  df_train_ip = token.texts_to_matrix(train_cont,mode='tfidf')
  df_test_ip = token.texts_to_matrix(test_cont,mode='tfidf')

  df_train_ip = df_train_ip/np.amax(np.absolute(df_train_ip))
  df_test_ip = df_test_ip/np.amax(np.absolute(df_test_ip))

  df_train_ip = df_train_ip  - np.mean(df_train_ip)
  df_test_ip = df_test_ip - np.mean(df_test_ip)

  df_train_op = to_categorical(df['CLASS'].iloc[train_idx])
  df_test_op = to_categorical(df['CLASS'].iloc[test_idx])

  model = Sequential()
  model.add(Dense(512,input_shape=(1000,)))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(2))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

  model.fit(df_train_ip,df_train_op,epochs=10,batch_size=16)

  scores= model.evaluate(df_test_ip,df_test_op)
  return scores



In [ ]:
kfold = StratifiedKFold(n_splits=5)
splits = kfold.split(df,df['CLASS'])
cvscores =[]
for train_idx,test_idx in splits:
  scores = train_test(train_idx,test_idx)
  cvscores.append(scores[1] * 100)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7433 - loss: 0.6232
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9453 - loss: 0.2591
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9645 - loss: 0.1354
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9791 - loss: 0.0856
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9853 - loss: 0.0674
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9893 - loss: 0.0463
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9971 - loss: 0.0297
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9957 - loss: 0.0293
Epoch 9/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9955 - loss: 0.0233
Epoch 10/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9944 - loss: 0.0207
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9578 - loss: 0.1516
Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7797 - loss: 0.6296
Epoch 2/10


In [ ]:
import keras
from keras.utils import to_categorical
